In [1]:
import argparse
import os
import glob
import subprocess
import logging
import sys
import os
from netCDF4 import Dataset
from shutil import copyfile
import zipfile
from pymongo import MongoClient
import json
import numpy
import datetime
from gbdhidro import utilcf

In [2]:
#HERE = os.path.abspath(os.path.dirname(__file__))
INPUT_FILE = '/media/jairo/Dados/Jairo/Projetos/Samuel data/git/GBD-Hidro/src/test/output/hobo_ua_003_64/EHP02039.nc'
rootgrp = Dataset(INPUT_FILE, 'r')

In [3]:
rootgrp.dimensions

{'time': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 458,
 'name_strlen': <class 'netCDF4._netCDF4.Dimension'>: name = 'name_strlen', size = 23}

In [4]:
rootgrp.groups

{}

In [5]:
rootgrp.variables

{'time': <class 'netCDF4._netCDF4.Variable'>
 float64 time(time)
     long_name: time of measurement
     standard_name: time
     units: seconds since 1970-01-01 00:00:00 UTC
     axis: T
     calendar: gregorian
     comment: 
 unlimited dimensions: time
 current shape = (458,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float32 lat()
     _FillValue: -999.9
     long_name: station latitude
     standard_name: latitude
     units: degrees_north
     axis: Y
     valid_min: -90.0
     valid_max: 90.0
     comment: 
 unlimited dimensions: 
 current shape = ()
 filling on,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float32 lon()
     _FillValue: -999.9
     long_name: station longitude
     standard_name: longitude
     units: degrees_east
     axis: X
     valid_min: -180.0
     valid_max: 180.0
     comment: 
 unlimited dimensions: 
 current shape = ()
 filling on,
 'alt': <class 'netCDF4._netCDF4.Variable'>
 float32 al

In [6]:
# global attributes
rootgrp.__dict__

{'title': 'Observed precipitation raw data',
 'summary': 'Observed precipitation data',
 'Conventions': 'CF-1.7',
 'institution': 'Universidade Federal de Pelotas',
 'instrument': 'HOBO Pendant Event 64K (UA-003-64)',
 'id': '',
 'comment': '',
 'creator_name': 'Grupo de Pesquisa em Hidrologia e Modelagem Hidrológica em Bacias Hidrográficas',
 'creator_url': 'https://wp.ufpel.edu.br/hidrologiaemodelagemhidrologica/',
 'geospatial_lat_units': 'degrees_north',
 'geospatial_lon_units': 'degrees_east',
 'geospatial_vertical_min': 0.0,
 'geospatial_vertical_max': 0.0,
 'geospatial_vertical_units': 'm',
 'geospatial_vertical_resolution': 1.0,
 'geospatial_vertical_positive': '',
 'keywords': ['lwe_thickness_of_precipitation_amount', 'mm', 'EH-P02'],
 'key_variables': 'precipitation',
 'history': '(20200706T185459Z) Created with HOBO Pendant Event Data Logger (UA-003-64) to NetCDF conversion tool v.0.0.1',
 'date_created': '20200706T185459Z',
 'geospatial_lat_min': -31.425,
 'geospatial_lat_m

In [7]:
rootgrp.ncattrs()

['title',
 'summary',
 'Conventions',
 'institution',
 'instrument',
 'id',
 'comment',
 'creator_name',
 'creator_url',
 'geospatial_lat_units',
 'geospatial_lon_units',
 'geospatial_vertical_min',
 'geospatial_vertical_max',
 'geospatial_vertical_units',
 'geospatial_vertical_resolution',
 'geospatial_vertical_positive',
 'keywords',
 'key_variables',
 'history',
 'date_created',
 'geospatial_lat_min',
 'geospatial_lat_max',
 'geospatial_lon_min',
 'geospatial_lon_max',
 'time_coverage_start',
 'time_coverage_end',
 'time_coverage_duration',
 'database_uuid']

In [13]:
# Converte os atribuito globais do NetCDF para formatos mais adequados do MongoDB
def convert_netcdf_attributes_to_mongo(dataset):
    attributes = dataset.__dict__.copy()    
    d = {}
    for key, value in attributes.items():
        # Convert numpy type to native python
        if type(value).__module__ == numpy.__name__:
            value = value.item()
        # Convert time converage start/end to native mongodb
        if key == 'time_coverage_start':
            #value = datetime.datetime.strptime(value, '%Y%m%dT%H%M%SZ')
            value = utilcf.iso8601_to_datetime(value)
        if key == 'time_coverage_end':
            value = utilcf.iso8601_to_datetime(value)
        if key == 'time_coverage_duration':
            # Nao tem tipo de intervalo de tempo definido no mongodb. Guardar o intervalor em segundos
            # Calcula a partir da data de incio e fim pois e mais facil dq converter valor inserido
            # no netcdf. Nao sei direito com lidar com isso ainda
            start = utilcf.iso8601_to_datetime(attributes['time_coverage_start'])
            end = utilcf.iso8601_to_datetime(attributes['time_coverage_end'])
            interval = (end - start).total_seconds()
            value = int(interval)
            
        d.update({key: value})
    return d

metadata = convert_netcdf_attributes_to_mongo(rootgrp)
metadata

{'title': 'Observed precipitation raw data',
 'summary': 'Observed precipitation data',
 'Conventions': 'CF-1.7',
 'institution': 'Universidade Federal de Pelotas',
 'instrument': 'HOBO Pendant Event 64K (UA-003-64)',
 'id': '',
 'comment': '',
 'creator_name': 'Grupo de Pesquisa em Hidrologia e Modelagem Hidrológica em Bacias Hidrográficas',
 'creator_url': 'https://wp.ufpel.edu.br/hidrologiaemodelagemhidrologica/',
 'geospatial_lat_units': 'degrees_north',
 'geospatial_lon_units': 'degrees_east',
 'geospatial_vertical_min': 0.0,
 'geospatial_vertical_max': 0.0,
 'geospatial_vertical_units': 'm',
 'geospatial_vertical_resolution': 1.0,
 'geospatial_vertical_positive': '',
 'keywords': ['lwe_thickness_of_precipitation_amount', 'mm', 'EH-P02'],
 'key_variables': 'precipitation',
 'history': '(20200706T185459Z) Created with HOBO Pendant Event Data Logger (UA-003-64) to NetCDF conversion tool v.0.0.1',
 'date_created': '20200706T185459Z',
 'geospatial_lat_min': -31.424999237060547,
 'geos

In [24]:
MONGODB_URL = '127.0.0.1:27017'
DATABASE = 'gbdhidro'
TABLE = 'index'


def insert_entry_db(entry):
    client = MongoClient(MONGODB_URL)
    gbd=client[DATABASE]
    index = gbd[TABLE]
    index.insert_one(entry)
    client.close()

In [25]:
insert_entry_db(metadata)

DuplicateKeyError: E11000 duplicate key error collection: gbdhidro.index index: _id_ dup key: { : ObjectId('5f05ff6bf08895581878349a') }

In [20]:
rootgrp.close()

RuntimeError: NetCDF: Not a valid ID